In [0]:
%sql
select * from catalog1_we47.default.account

![](/Workspace/Users/infoblisstech@gmail.com/databricks-code-repo/6_lakeflow_pipelines/autoloader_file_ingestion_usecase1.png)

###Auto Loader is Databricks
**Auto Loader is Databricks’ cloud-native file ingestion engine for ingesting new files incrementally from object storage.**

Supported Sources:
- AWS S3
- Azure ADLS Gen2
- Google Cloud Storage (GCS)

Modes:
- Directory listing - Directory listing scans storage paths to detect new files (This works in free edition)
- File notification - Processes files as soon as they arrive at scale (This will not work in free edition because the cloud storage event trigger can't control/trigger Databricks LF Ingestion)

**Directory listing**
1. Spark lists the Cloud directory (pull model)
2. Detects new files
3. Infers schema / evolves if needed
4. Copy the file(s) & store the schema info in a schema file, so further schema inference is not needed.
5. Updates checkpoint (file1 is processed...)
6. Waits for next trigger of the Lakeflow pipeline

**File Notification** (we will see it in the cloud databricks version)
1. Cloud storage emits file-create event (S3 Event, ADLS Event Grid, GCS Pub/Sub)
2. Event is delivered to Databricks queue
3. Auto Loader receives notification (push model)
4. New file is registered
5. Infers schema / evolves if needed
6. Copy the file(s) & store the schema info in a schema file, so further schema inference is not needed.
7. Updates checkpoint (file1 is processed...)
8. Stream stays idle until next event arrives

**To perform schema evolution, we have to use the below properties:**<br>
**Read side:** <br>
.option("cloudFiles.schemaEvolutionMode","addNewColumns")<br>
**Write side:** <br>
.option("mergeSchema", "true")<br>

In [0]:
ckptlocation="/Volumes/catalog3_we47/schema3_we47/datalake/bronze/streamwrite13/_checkpoint"
schemalocation="/Volumes/catalog3_we47/schema3_we47/datalake/bronze/streamwrite54/_schema"
cloudsrc="/Volumes/catalog3_we47/schema3_we47/datalake/source2/"#s3/gcs/adls/blob storage path
bronzetgt="/Volumes/catalog3_we47/schema3_we47/datalake/bronze/streamwrite13/"
df1=spark.readStream.format("cloudFiles")\
.option("cloudFiles.format","csv")\
.option("cloudFiles.maxFilesPerTrigger",5)\
.option("cloudFiles.inferColumnTypes",True)\
.option("cloudFiles.schemaEvolutionMode","addNewColumns")\
.option("checkpointLocation", ckptlocation)\
.option("cloudFiles.schemaLocation", schemalocation)\
.option("header",True)\
.load(cloudsrc)#this can be s3/adls/gcs
#.option("cloudFiles.useNotifications", "true") (Remove this option to enable directory listing)
#maxFilesPerTrigger - this property help spark to process howmany files in an iteration to control the resource utilization (all files will be processed ultimately)

In [0]:
#realtime trigger is not possible in free serverless
df1.writeStream.trigger(availableNow=True)\
.option("checkpointLocation", ckptlocation)\
.option("cloudFiles.schemaLocation", schemalocation)\
.option("mergeSchema", "true") \
.start(bronzetgt)
#.option("mergeSchema", "true") \

In [0]:
spark.read.format("delta").load(f"/Volumes/catalog3_we47/schema3_we47/datalake/bronze/streamwrite13").orderBy("city_name").show(100)